In [1]:
# Importing all the required Libraries 
import unidecode 
import pandas as pd 
import re
import time 
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
from autocorrect import Speller 
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords 
from nltk import word_tokenize 
import string 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
import spacy
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
df_questions = pd.read_csv(r"D:\Yash-USA\Northeastern University\Applied Artificial Intelligence\Quarter-4 Fall'22\Fall Term - AB\INT 6940 Experiential Learning Projects for Professionals\Repurpost\Dataset\Questions.csv", encoding="ISO-8859-1")
df_questions.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [4]:
df_questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264216 entries, 0 to 1264215
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Id            1264216 non-null  int64  
 1   OwnerUserId   1249762 non-null  float64
 2   CreationDate  1264216 non-null  object 
 3   ClosedDate    55959 non-null    object 
 4   Score         1264216 non-null  int64  
 5   Title         1264216 non-null  object 
 6   Body          1264216 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 67.5+ MB


In [5]:
# Dropping columns that are not necessary for repurpost project
df_questions.drop(columns=['OwnerUserId', 'CreationDate', 'ClosedDate'], inplace=True)
df_questions.head()

,Id,Score,Title,Body
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [8]:
df_tags = pd.read_csv(r"D:\Yash-USA\Northeastern University\Applied Artificial Intelligence\Quarter-4 Fall'22\Fall Term - AB\INT 6940 Experiential Learning Projects for Professionals\Repurpost\Dataset\Tags.csv", encoding="ISO-8859-1")
df_tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [9]:
df_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750994 entries, 0 to 3750993
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Id      int64 
 1   Tag     object
dtypes: int64(1), object(1)
memory usage: 57.2+ MB


In [10]:
# The same id is having more than one tag
# to combine this data with the questions, we need to combine the all the tags belonging to an id
# and then join with questions dataframe to avoid cartesion product of the join

# Convert Tag column datatype to string for grouping
df_tags['Tag'] = df_tags['Tag'].astype(str)

# Combining the tags for a particular Id separated by pipe '|' delimiter
df_tags_grouped = pd.DataFrame(df_tags.groupby('Id')['Tag'].apply(lambda tag: '|'.join(tag))).reset_index()
df_tags_grouped.head()

,Id,Tag
0,80,flex|actionscript-3|air
1,90,svn|tortoisesvn|branch|branching-and-merging
2,120,sql|asp.net|sitemap
3,180,algorithm|language-agnostic|colors|color-space
4,260,c#|.net|scripting|compiler-construction


In [11]:
# Check the rows and columns of df_tags_grouped dataframe
df_tags_grouped.shape

(1264216, 2)

In [12]:
# Check the rows and columns of df_question dataframe
df_questions.shape

(1264216, 4)

In [13]:
# Combine the questions and tags for next steps
df_stackoverflow = df_questions.merge(df_tags_grouped, on='Id')
df_stackoverflow.head()

,Id,Score,Title,Body,Tag
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,flex|actionscript-3|air
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,svn|tortoisesvn|branch|branching-and-merging
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql|asp.net|sitemap
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,algorithm|language-agnostic|colors|color-space
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,c#|.net|scripting|compiler-construction


In [14]:
del df_questions
del df_tags
del df_tags_grouped

In [15]:
df_stackoverflow['Tags'] = df_stackoverflow['Tag'].apply(lambda x: x.split('|'))
df_stackoverflow.head()

,Id,Score,Title,Body,Tag,Tags
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,flex|actionscript-3|air,"[flex, actionscript-3, air]"
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,svn|tortoisesvn|branch|branching-and-merging,"[svn, tortoisesvn, branch, branching-and-merging]"
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql|asp.net|sitemap,"[sql, asp.net, sitemap]"
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,algorithm|language-agnostic|colors|color-space,"[algorithm, language-agnostic, colors, color-s..."
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,c#|.net|scripting|compiler-construction,"[c#, .net, scripting, compiler-construction]"


In [18]:
tags_list = [tag for tags in df_stackoverflow[df_stackoverflow['Score'] > 5]['Tags'].values for tag in tags]

In [19]:
tag_counts = nltk.FreqDist(tags_list)
top_1000_tags = [tag[0] for tag in tag_counts.most_common(1000)]
top_1000_tags[0:20]

['c#',
 'java',
 'javascript',
 'android',
 'python',
 'c++',
 'php',
 'jquery',
 '.net',
 'ios',
 'html',
 'css',
 'c',
 'iphone',
 'objective-c',
 'ruby-on-rails',
 'sql',
 'asp.net',
 'mysql',
 'ruby']

In [20]:
# Define function to take list of tags as input and create a new list only based
# on tags that are part of the top 1000 tags list
def keep_top_tags(tags):
  tags_list = []
  for tag in tags:
    if tag in top_1000_tags:
      tags_list.append(tag)

  if (len(tags_list) > 0):
    return tags_list
  else:
    return None

In [21]:
df_stackoverflow['Tags'] = df_stackoverflow['Tags'].apply(lambda tags: keep_top_tags(tags))

In [22]:
df_stackoverflow.head()

,Id,Score,Title,Body,Tag,Tags
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,flex|actionscript-3|air,"[flex, actionscript-3]"
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,svn|tortoisesvn|branch|branching-and-merging,"[svn, tortoisesvn, branch]"
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql|asp.net|sitemap,"[sql, asp.net]"
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,algorithm|language-agnostic|colors|color-space,"[algorithm, language-agnostic, colors]"
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,c#|.net|scripting|compiler-construction,"[c#, .net, scripting, compiler-construction]"


In [23]:
df_stackoverflow = df_stackoverflow[df_stackoverflow['Score'] > 5].dropna()

In [24]:
# Drop extra columns.
df_stackoverflow.drop(columns=['Tag'], inplace=True)
df_stackoverflow.head()

,Id,Score,Title,Body,Tags
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3]"
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch]"
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net]"
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,"[algorithm, language-agnostic, colors]"
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net, scripting, compiler-construction]"


In [25]:
df_stackoverflow.shape

(71237, 5)

In [26]:
df_stackoverflow.isna().sum()

Id       0
Score    0
Title    0
Body     0
Tags     0
dtype: int64

### Data Cleaning and pre-processing:

In [27]:
df_stackoverflow.head()

,Id,Score,Title,Body,Tags
0,80,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[flex, actionscript-3]"
1,90,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[svn, tortoisesvn, branch]"
2,120,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net]"
3,180,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,"[algorithm, language-agnostic, colors]"
4,260,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net, scripting, compiler-construction]"


In [28]:
#Cleaning HTML tags from the title and body text
def clean_html(html):

    # parse html content
    soup = BeautifulSoup(html, "html.parser")

    for data in soup(['style', 'script', 'code', 'a']):
        # Remove tags
        data.decompose()

    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

In [29]:
# To remove HTML first and apply it directly to the source text column.
df_stackoverflow['Body'] = df_stackoverflow['Body'].apply(lambda x: clean_html(x))

In the below code chunk, the data has been passed through a function which cleans the data by making the text into lower case and by removing the line breaks, punctuations, stop words and numbers. And also, stemming and lemmatization is performed. 

In [30]:
# Load spacy
nlp = spacy.load('en_core_web_sm')

def clean_string(text, stem="None"):

    final_string = ""

    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub(r'\n', '', text)

    # Remove puncuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    useless_words = useless_words + ['hi', 'im']

    text_filtered = [word for word in text if not word in useless_words]

    # Remove numbers
    text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]

    # Stem or Lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer() 
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string

C:\Users\User\anaconda3\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.4.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [31]:
# Next apply the clean_string function to the title text
df_stackoverflow['title_clean'] = df_stackoverflow['Title'].apply(lambda x: clean_string(x, stem='Stem'))

In [32]:
df_stackoverflow.head()

,Id,Score,Title,Body,Tags,title_clean
0,80,26,SQLStatement.execute() - multiple queries in o...,I've written a database generation script in a...,"[flex, actionscript-3]",sqlstatementexecut multipl queri one statement
1,90,144,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...,"[svn, tortoisesvn, branch]",good branch merg tutori tortoisesvn
2,120,21,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...,"[sql, asp.net]",aspnet site map
3,180,53,Function for creating color wheels,This is something I've pseudo-solved many time...,"[algorithm, language-agnostic, colors]",function creat color wheel
4,260,49,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,"[c#, .net, scripting, compiler-construction]",ad script function net applic


The body text contains punctuations which should not be removed because this attribute helps the models to understand and makes sense with the tags attribute to run the model efficicnetly. So, the following data clean code is rpeated but without punctuiation removal in the 'Body' column

In [35]:
# Load spacy
nlp = spacy.load('en_core_web_sm')

def clean_string_body(text, stem="None"):

    final_string = ""

    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub(r'\n', '', text)

    # Remove puncuation
    #translator = str.maketrans('', '', string.punctuation)
    #text = text.translate(translator)

    # Remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    useless_words = useless_words + ['hi', 'im']

    text_filtered = [word for word in text if not word in useless_words]

    # Remove numbers
    text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]

    # Stem or Lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer() 
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string

In [36]:
# Next apply the clean_string function to the body text
df_stackoverflow['body_clean'] = df_stackoverflow['Body'].apply(lambda x: clean_string_body(x, stem='Stem'))

In [37]:
df_stackoverflow.head()

,Id,Score,Title,Body,Tags,title_clean,body_clean
0,80,26,SQLStatement.execute() - multiple queries in o...,I've written a database generation script in a...,"[flex, actionscript-3]",sqlstatementexecut multipl queri one statement,i'v written databas gener script want execut a...
1,90,144,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...,"[svn, tortoisesvn, branch]",good branch merg tutori tortoisesvn,realli good tutori explain apach subversion? b...
2,120,21,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...,"[sql, asp.net]",aspnet site map,anyon got experi creat sql-base asp.net site-m...
3,180,53,Function for creating color wheels,This is something I've pseudo-solved many time...,"[algorithm, language-agnostic, colors]",function creat color wheel,someth i'v pseudo-solv mani time never quit fo...
4,260,49,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,"[c#, .net, scripting, compiler-construction]",ad script function net applic,"littl game written c#. use databas back-end. ,..."


In [52]:
# Saving the cleaned dataset

In [58]:
df_stackoverflow.to_csv("StackoverFlow_Final",index = False)

In [63]:
import pandas as pd
import os.path
df_stackoverflow.to_csv(os.path.join(r"C:\Users\User\OneDrive\Desktop",'StackoverFlow_Final.csv'))

#### Tokenization

In [39]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_stackoverflow['body_clean'])

In [40]:
# check unique words count
len(tokenizer.word_index)

76194

In [41]:
# check unique words count
vocab_size = len(tokenizer.word_index) + 1
vocab_size

76195

In [42]:
sequences = tokenizer.texts_to_sequences(df_stackoverflow['body_clean'])

In [43]:
for i in range(20):
    print(df_stackoverflow['body_clean'][i], '\n'), print(sequences[i])

i'v written databas gener script want execut application: execut adob air use follow methods: error generated, howev exists. seem look first queri (up semicolon- remov it, queri fails). way call multipl queri one statement? 

[20, 433, 184, 71, 136, 9, 211, 239, 211, 3456, 4033, 1, 22, 799, 16, 2180, 246, 1340, 30, 26, 53, 187, 461, 4298, 228, 31, 187, 1459, 11, 28, 251, 187, 12, 362]
realli good tutori explain apach subversion? better specif tortoisesvn client. 

[110, 143, 739, 340, 1016, 4435, 165, 194, 4299, 190]
anyon got experi creat sql-base asp.net site-map providers? i'v got default xml file work properli menu sitemappath controls, i'll need way user site creat modifi page dynamically. need tie page view permiss standard membership system well. 

[82, 185, 604, 21, 240, 146, 396, 124, 271, 273, 5461, 20, 185, 193, 215, 10, 6, 833, 563, 35622, 2348, 614, 13, 11, 35, 271, 21, 402, 54, 2395, 13, 2091, 54, 76, 901, 303, 2909, 119, 147]
someth i'v pseudo-solv mani time never quit f

#### MultiLabel Binarization

In [44]:
#separating and storing tags into a variable
tags = df_stackoverflow['Tags']

In [45]:
tags

0                                [flex, actionscript-3]
1                            [svn, tortoisesvn, branch]
2                                        [sql, asp.net]
3                [algorithm, language-agnostic, colors]
4          [c#, .net, scripting, compiler-construction]
                               ...                     
1262915                     [python, pandas, dataframe]
1263065                              [python, iterator]
1263399                                            [f#]
1263454                                           [c++]
1264205        [haskell, types, functional-programming]
Name: Tags, Length: 71237, dtype: object

In [46]:

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_stackoverflow['Tags'])
binarized_tags = multilabel_binarizer.transform(df_stackoverflow['Tags'])

In [47]:
#tags after binarization are stored as an array
binarized_tags

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [55]:
multilabel_binarizer.classes_

array(['.htaccess', '.net', '.net-2.0', '.net-3.5', '.net-4.0',
       '.net-4.5', '32bit-64bit', '3d', '64bit', 'abstract-class',
       'accessibility', 'actionbarsherlock', 'actionscript-3',
       'active-directory', 'activerecord', 'adb', 'ado.net', 'ajax',
       'akka', 'algorithm', 'alignment', 'amazon-ec2', 'amazon-s3',
       'amazon-web-services', 'android', 'android-5.0-lollipop',
       'android-actionbar', 'android-activity', 'android-animation',
       'android-asynctask', 'android-canvas', 'android-edittext',
       'android-emulator', 'android-fragments', 'android-gradle',
       'android-intent', 'android-layout', 'android-listview',
       'android-manifest', 'android-ndk', 'android-studio',
       'android-support-library', 'android-viewpager', 'android-widget',
       'angular-ui-router', 'angular2', 'angularjs',
       'angularjs-directive', 'angularjs-scope', 'animation',
       'annotations', 'ant', 'aop', 'apache', 'apache-spark', 'api',
       'app-store', 'ap

In [48]:
#binarized tags as a dataframe
binarized_tags = pd.DataFrame(binarized_tags, columns=multilabel_binarizer.classes_)
binarized_tags

,.htaccess,.net,.net-2.0,.net-3.5,.net-4.0,.net-4.5,32bit-64bit,3d,64bit,abstract-class,...,xmlhttprequest,xna,xpath,xsd,xslt,yii,youtube,youtube-api,zend-framework,zip
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71232,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71233,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71234,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71235,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#TFIDF
#from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
#tf idf
#tf_idf = TfidfVectorizer()
#applying tf idf to body text
#body_tf = tf_idf.fit_transform(df_stackoverflow['body_clean'])

In [ ]:
#print(body_tf)

In [ ]:
#applying tf idf to title text
#title_tf = tf_idf.fit_transform(df_stackoverflow['title_clean'])

In [ ]:
#print(title_tf)

In [ ]:
#df_stackoverflow.head()

In [49]:
df_stackoverflow.head()

,Id,Score,Title,Body,Tags,title_clean,body_clean
0,80,26,SQLStatement.execute() - multiple queries in o...,I've written a database generation script in a...,"[flex, actionscript-3]",sqlstatementexecut multipl queri one statement,i'v written databas gener script want execut a...
1,90,144,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...,"[svn, tortoisesvn, branch]",good branch merg tutori tortoisesvn,realli good tutori explain apach subversion? b...
2,120,21,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...,"[sql, asp.net]",aspnet site map,anyon got experi creat sql-base asp.net site-m...
3,180,53,Function for creating color wheels,This is something I've pseudo-solved many time...,"[algorithm, language-agnostic, colors]",function creat color wheel,someth i'v pseudo-solv mani time never quit fo...
4,260,49,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,"[c#, .net, scripting, compiler-construction]",ad script function net applic,"littl game written c#. use databas back-end. ,..."


##### Tf-IDF Vectorization

In [66]:
#TfIDF

In [50]:
tfidf = TfidfVectorizer(analyzer='word', max_features=10000, ngram_range=(1,3), stop_words='english')
X = tfidf.fit_transform(df_stackoverflow['body_clean'])

In [64]:
X.shape, binarized_tags.shape

((71237, 10000), (71237, 1000))

In [51]:
print(X)

  (0, 8951)	0.17399953490573608
  (0, 9448)	0.2513970614781061
  (0, 7268)	0.2600945007707953
  (0, 7884)	0.15403682619365996
  (0, 5331)	0.13855705700632032
  (0, 9526)	0.07622697705074473
  (0, 2913)	0.20016334739059322
  (0, 6874)	0.13702741804666824
  (0, 7351)	0.26963115125518416
  (0, 6554)	0.42699137504078927
  (0, 4851)	0.09286106580205387
  (0, 2828)	0.19767474859444875
  (0, 3777)	0.13533506416549715
  (0, 3445)	0.22597755290773033
  (0, 2636)	0.09577809319934635
  (0, 5196)	0.17560684364799428
  (0, 3184)	0.0909842774713316
  (0, 8850)	0.05299753286330783
  (0, 202)	0.2631821466518189
  (0, 166)	0.25420280239605997
  (0, 492)	0.13597990332727664
  (0, 2803)	0.2738348783247547
  (0, 9416)	0.07595533220593954
  (0, 7258)	0.1351960941660137
  (0, 3424)	0.11756831485915521
  :	:
  (71236, 2149)	0.18438402183407
  (71236, 8683)	0.35552134964932997
  (71236, 7550)	0.07670063965952209
  (71236, 4505)	0.10063260801939336
  (71236, 6298)	0.09935784097661474
  (71236, 3847)	0.15790881

#### Train and Test Split for Model Building

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, binarized_tags, test_size = 0.2, random_state = 0)

#### Model Building